In [37]:
#Libraries
from lxml import html
import requests
import unicodecsv as csv
import argparse
import pandas as pd
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from fake_useragent import UserAgent
import re
from datetime import datetime
import time
import numpy as np

### Getting transcript links

In [38]:
url="https://seekingalpha.com/earnings/earnings-call-transcripts/"

page = 1
link_store = []

In [39]:
while(page <= 10):
    new_url = url + str(page)
    page = page + 1
    html_content = requests.get(new_url).text
    
    # Parse the html content
    soup_link = soup(html_content, "lxml")

    for link in soup_link.findAll('a',{"class":"dashboard-article-link"}):
            link_href = link.get('href')
            combined_link = 'https://seekingalpha.com' + link_href
            link_store.append(combined_link)


In [61]:
"""
link_store.append("https://seekingalpha.com/article/4393745-newmont-corporation-nem-ceo-tom-palmer-hosts-2021-investor-update-call-transcript")
link_store.append("https://seekingalpha.com/article/4393736-mongodb-inc-mdb-ceo-dev-ittycheria-on-q3-2021-results-earnings-call-transcript")
link_store.append("https://seekingalpha.com/article/4393631-barnes-noble-education-inc-bned-ceo-mike-huseby-on-q2-2021-results-earnings-call-transcript")
link_store.append("https://seekingalpha.com/article/4393377-lightinthebox-holding-co-ltd-s-litb-ceo-jian-on-q3-2020-results-earnings-call-transcript")
link_store.append("https://seekingalpha.com/article/4392903-idt-corporations-idt-ceo-shmuel-jonas-on-q1-2021-results-earnings-call-transcript")
link_store.append("https://seekingalpha.com/article/4392859-union-pacific-corporation-unp-ceo-lance-fritz-presents-credit-suisse-8th-annual-virtual")
link_store.append("https://seekingalpha.com/article/4392655-sarepta-therapeutics-inc-srpt-presents-evercore-isi-healthconx-conference-transcript")
link_store.append("https://seekingalpha.com/article/4392571-regeneron-pharmaceuticals-incs-regn-management-presents-piper-sandler-32nd-annual-virtual")
link_store.append("https://seekingalpha.com/article/4392384-twitter-inc-twtr-presents-bofa-2020-leveraged-finance-virtual-conference-transcript")
link_store.append("https://seekingalpha.com/article/4391728-laix-inc-s-laix-ceo-yi-wang-on-q3-2020-results-earnings-call-transcript")
"""

'\nlink_store.append("https://seekingalpha.com/article/4393745-newmont-corporation-nem-ceo-tom-palmer-hosts-2021-investor-update-call-transcript")\nlink_store.append("https://seekingalpha.com/article/4393736-mongodb-inc-mdb-ceo-dev-ittycheria-on-q3-2021-results-earnings-call-transcript")\nlink_store.append("https://seekingalpha.com/article/4393631-barnes-noble-education-inc-bned-ceo-mike-huseby-on-q2-2021-results-earnings-call-transcript")\nlink_store.append("https://seekingalpha.com/article/4393377-lightinthebox-holding-co-ltd-s-litb-ceo-jian-on-q3-2020-results-earnings-call-transcript")\nlink_store.append("https://seekingalpha.com/article/4392903-idt-corporations-idt-ceo-shmuel-jonas-on-q1-2021-results-earnings-call-transcript")\nlink_store.append("https://seekingalpha.com/article/4392859-union-pacific-corporation-unp-ceo-lance-fritz-presents-credit-suisse-8th-annual-virtual")\nlink_store.append("https://seekingalpha.com/article/4392655-sarepta-therapeutics-inc-srpt-presents-evercore

#### Feature Collection Functions

##### Title

In [42]:
def getTitle(transcript_soup):
    title = str(transcript_soup.findAll("title")[0])
    title = title.replace('<title>','')
    title = title.replace('</title>','')
    return title

##### Ticker

In [43]:
def getTicker(title):
    ticker = re.findall(r'\((.*?)\)',title)[0]
    return ticker

##### Company

In [44]:
def getCompany(title):
    company = re.findall(r'^([^.]*).*',title)[0]
    return company

##### Date

In [45]:
def getDate(transcript_soup):
    date = str(transcript_soup.findAll('meta',{'name':'last-modified'})[0])
    date =date.replace('<meta content="','')
    date = date.split(' ')[0]
    return date

##### Call Time

In [46]:
def time_converter(time_string):
    if('PM' in time_string):
        number, remaining = time_string.split(':')
        remaining = remaining.replace(' PM','')
        changed_number = str(int(number) + 12)
        return (changed_number + ':' + remaining)
    else:
        time_string = time_string.replace(' AM','')
        return time_string

In [47]:
def getTime(transcript_soup):
    time_description = str(transcript_soup.findAll('meta',{'name':'description'}))
    time = (re.search(r'\d?\d:\d\d[\s]\w[.]?\w', time_description))[0]
    time = time_converter(time)
    return time

##### Presentation Speech

In [48]:
def getPresentationSpeech(transcript_soup):
    overall_text = transcript_soup.findAll('p')
    split_text = str(overall_text).split('question-answer-session')
    presentation_speech = split_text[0]
    return presentation_speech

##### Question Answer Speech

In [49]:
def getQuestionAnswerSpeech(transcript_soup):
    overall_text = transcript_soup.findAll('p')
    split_text = str(overall_text).split('question-answer-session')
    question_answer_speech = split_text[1]
    return question_answer_speech

##### Company Participant Names

In [50]:
def getCompanyParticipants(transcript_soup):
    merged_p = ''
    for p in transcript_soup.findAll('p', {'class':'p p1'}):
        merged_p = merged_p + ' ' + str(p)
    company_split = merged_p.split('p1"><strong>Conference')
    company_participant_half = company_split[0]
    company_participant_list = re.findall(r'\S+ \S+ -',company_participant_half)
    participant_index = 0
    for participant in company_participant_list:
        participant = participant.replace('p1">','')
        company_participant_list[participant_index] = participant.replace(' -', '')
        participant_index = participant_index + 1
    company_participant_string = ''
    for company_participant in company_participant_list:
        if company_participant_string == "":
            company_participant_string = company_participant
        else:
            company_participant_string = company_participant_string + '.+.' + company_participant
    return company_participant_string

##### Noncompany Participant Names

In [51]:
def getNoncompanyParticipants(transcript_soup):
    merged_p = ''
    for p in transcript_soup.findAll('p', {'class':'p p1'}):
        merged_p = merged_p + ' ' + str(p)
    company_split = merged_p.split('p1"><strong>Conference')
    if(len(company_split) > 1):
        other_participant_half = company_split[1]

        other_participant_list = re.findall(r'\S+ \S+ -',other_participant_half)
        participant_index_noncompany = 0
        for participant in other_participant_list:
            participant = participant.replace('p1">','')
            other_participant_list[participant_index_noncompany] = participant.replace(' -', '')
            participant_index_noncompany = participant_index_noncompany + 1
        noncompany_participant_string = ''
        for noncompany_participant in other_participant_list:
            if noncompany_participant_string == "":
                noncompany_participant_string = noncompany_participant
            else:
                noncompany_participant_string = noncompany_participant_string + '.+.' + noncompany_participant
        return noncompany_participant_string
    else:
        return ''

##### Company Participant Type

In [52]:
def getCompanyParticipantType(transcript_soup):
    merged_p = ''
    for p in transcript_soup.findAll('p', {'class':'p p1'}):
        merged_p = merged_p + ' ' + str(p)
    company_split = merged_p.split('p1"><strong>Conference')
    company_participant_half = company_split[0]
    title_list = re.findall(r'- \S+[^<]*',company_participant_half)
    title_index = 0
    for title in title_list:
        title = title.replace(',','')
        title_list[title_index] = title.replace('- ', '')
        title_index = title_index + 1
    title_list_string = ''
    for title in title_list:
        title = title.replace(',','')
        if title_list_string == "":
            title_list_string = title
        else:
            title_list_string = title_list_string + '.+.' + title
    return title_list_string

##### Noncompany Participant Organizations

In [53]:
def getNoncompanyParticipantOrganizations(transcript_soup):
    merged_p = ''
    for p in transcript_soup.findAll('p', {'class':'p p1'}):
        merged_p = merged_p + ' ' + str(p)
    company_split = merged_p.split('p1"><strong>Conference')
    if(len(company_split)>1):
        other_participant_half = company_split[1]

        organization_list = re.findall(r'- \S+[^<]*',other_participant_half)

        title_index_organization = 0
        for organization in organization_list:
            organization_list[title_index_organization] = organization.replace('- ', '')
            title_index_organization = title_index_organization + 1
        participant_organization_string = ''

        for organization in organization_list:
            organization = organization.replace(',','')
            if participant_organization_string == "":
                participant_organization_string = organization
            else:
                participant_organization_string = participant_organization_string + '.+.' + organization
        return participant_organization_string
    else:
        return ''

In [54]:
transcript_scrape_column_list = ['ticker','company','title','date','call_time','presentation_speech','question_answer_speech','company_participant_list','noncompany_participant_list','company_participant_type','noncompany_participant_organization']

In [55]:
transcript_scrape = pd.DataFrame(columns=transcript_scrape_column_list)

In [56]:
transcript_scrape.head()

,ticker,company,title,date,call_time,presentation_speech,question_answer_speech,company_participant_list,noncompany_participant_list,company_participant_type,noncompany_participant_organization


In [57]:
'''
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate", 
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8", 
    "Dnt": "1", 
    "Host": "httpbin.org", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36", 
  }
'''

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
}

In [58]:
delays = [1,2,3,4,5]

In [59]:
for link in link_store:
    #Getting the transcript
    transcript_url = link
    transcript_content = requests.get(transcript_url, headers = headers).text
    transcript_soup = soup(transcript_content, "lxml")
    
    #Pulling features
    title = getTitle(transcript_soup)
    ticker = getTicker(title)
    company = getCompany(title)
    date = getDate(transcript_soup)
    call_time = getTime(transcript_soup)
    presentation_speech = getPresentationSpeech(transcript_soup)
    question_answer_speech = getQuestionAnswerSpeech(transcript_soup)
    company_participant_list = getCompanyParticipants(transcript_soup)
    noncompany_participant_list = getNoncompanyParticipants(transcript_soup)
    company_participant_type = getCompanyParticipantType(transcript_soup)
    noncompany_participant_organization = getNoncompanyParticipantOrganizations(transcript_soup)
    link_store.remove(link)
    delay = np.random.choice(delays)
    time.sleep(delay)
    
    #Storing into dataframe
    company_transcript_data = [ticker, company, title, date, call_time, presentation_speech, question_answer_speech, company_participant_list, noncompany_participant_list, company_participant_type, noncompany_participant_organization]
    transcript_scrape.loc[len(transcript_scrape)] = company_transcript_data

IndexError: list index out of range

In [60]:
transcript_scrape

,ticker,company,title,date,call_time,presentation_speech,question_answer_speech,company_participant_list,noncompany_participant_list,company_participant_type,noncompany_participant_organization


In [ ]:
transcript_scrape.to_excel("transcript_scrape_demo.xlsx")